[2020年- Default value of *some* fields in a mutable struct, like follow](https://discourse.julialang.org/t/default-value-of-some-fields-in-a-mutable-struct/33408)

```julia
mutable struct coly8
    x :: Int32
    y :: Float64
    z :: Bool
    t :: Int32
    u :: Float32
    v :: Bool
    coly8() = ( K = new(); K.x = 1 )
end
```


In [40]:
mutable struct coly8
    x :: Int32
    y :: Float64
    z :: Bool
    t :: Int32
    u :: Float32
    v :: Bool
    coly8() = ( K = new(); K.x = 1 )
end

In [42]:
using General.Aux

In [43]:
c8 = coly8()
c8_proper = propertynames(c8)
@logt c8 c8_proper


c8, Int64
  = 1

c8_proper, Tuple{}
  = ()


In [30]:
mutable struct coly_xy
    x :: Int32
    y :: Float64
    z :: Bool
    t :: Int32
    u :: Float32
    v :: Bool
    function coly_xy(;kwargs...)
        K = new()
        for (key, value) in kwargs
            setfield!(K, key, value)
        end
        return K
    end
end
coly_xy(t=Int32(13), v=true)  # 其他的给成了随机值

coly_xy(815291912, 0.0, false, 13, 0.0f0, true)

For more convenience you may want to add a convert call if the type doesn’t match.

如果类型不匹配，您可能希望添加一个 ` convert ` 调用以方便使用。

In [31]:
mutable struct coly_rand
    x :: Int32
    y :: Float64
    z :: Bool
    t :: Int32
    u :: Float32
    v :: Bool
    
    function coly_rand(;kwargs...)
        K = new()
        for (key, value) in kwargs
            field_type = typeof(getfield(K,key))
            setfield!(K, key, convert(field_type, value))
        end
        return K
    end    
end
coly_rand(x=2,y=2) # 其他随机

coly_rand(2, 2.0, false, 381, 7.3f-43, true)

In [33]:
@kwdef mutable struct coly_again
    x :: Int32
    y :: Float64
    z :: Bool
    t :: Int32
    u :: Float32
    v :: Bool
    
    function coly_again(;kwargs...)
        K = new()
        for (key, value) in kwargs
            field_type = typeof(getfield(K,key))
            setfield!(K, key, convert(field_type, value))
        end
        return K
    end    
end
coly_again(x=2,y=2)

UndefKeywordError: UndefKeywordError: keyword argument `z` not assigned

In [34]:
macro awesome(s)
    if s.head !== :struct 
        error("Not a struct def")
    end 

    ismutable = s.args[1]
    if !ismutable
        error("Not mutable")
    end
    
    name = s.args[2]
    body = s.args[3]    

    ctor = :(function $(name)(;kwargs...)
                K = new()
                for (key, value) in kwargs
                    field_type_key = typeof(getfield(K,key))
                    setfield!(K, key, convert(field_type_key, value))
                end
                return K
            end)
            
    newbody = [body.args; ctor]
    
    return Expr(s.head, ismutable, name, Expr(body.head, newbody...))       
end

@awesome (macro with 1 method)

In [36]:
@awesome mutable struct coly8
    x :: Int32
    y :: Float64
    z :: Bool
    t :: Int32
    u :: Float32
    v :: Bool
end
coly8(u=4.0)

coly8(142941568, 8.085512242926e-312, false, 381, 4.0f0, true)

The next step would be parsing a expression in a struct with predefined constants or use the x::Int = 42 syntax like in Parameters.jl. Truly limitless set of options :slight_smile: I would add also zeroing of not defined fields like most of languages do.

下一步是解析带有预定义常量的结构中的表达式，或者使用类似于Parameters.jl中的` x::Int = 42 `语法。我还会添加许多其他选项，例如像大多数语言那样将未定义的字段设置为零。

In [37]:
@kwdef struct Foo
    a::Int = 1         # specified default
    b::String          # required keyword
end
Foo(b="hi")
Foo()

UndefKeywordError: UndefKeywordError: keyword argument `b` not assigned